---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [111]:
import pandas as pd
import numpy as np
from datetime import datetime
doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line)

df = pd.Series(doc)
df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
#print(df.str.len())
#print(df.str.split().str.len())
df.str.split()[0]

['03/25/93', 'Total', 'time', 'of', 'visit', '(in', 'minutes):']

In [3]:
#how many lines we need to look up?
#print(len(df))

In [4]:
s = ' 6/2008; 12/2009'
s = '04/20/2009; 04/20/09; 4/20/09; 4/3/09'
import re
re.findall(r'(?:\s)\d{1,2}/\d{4}', s)

[]

In [5]:
df.str.extractall(r'(?<=[ ,.();:!?/])(?<!/)(?P<month>\d{1,2})[/-](?P<year>\d{4})[ ,.();:!?]{1,}\b')
df.str.extractall(r'^(?P<month>\d{1,2})[/-](?P<year>\d{4})[ ,.();:!?]{1,}\b')

,,month,year
,match,,
343,0,6,1998
345,0,10,1973
346,0,9,2005
348,0,12,2005
349,0,5,1987
350,0,5,2004
353,0,10,1997
355,0,3,1981
360,0,12,2008


In [89]:
#deal with str repressed as follows: "04/20/2009; 04/20/09; 4/20/09; 4/3/09"
pattern1_1 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<date>\d{1,2})[/-](?P<year>\d{2})\b').reset_index()
pattern1_1['year'] = '19'+pattern1_1['year']
pattern1_2 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<date>\d{1,2})[/-](?P<year>\d{4})\b').reset_index()
pattern1 = pd.concat([pattern1_1,pattern1_2])
pattern1.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates

#2009; 2010
#this should be considered with months expressed in strings such as:
#Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009;
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
#Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
#Feb 2009; Sep 2009; Oct 2010
len(pattern1)

125

In [90]:
#len(pattern1_2.append(pattern1_1)['level_0'].unique())

In [91]:
#Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; together 

pattern3 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[-.]*\s(?P<date>(?:\d{1,2}))[?:., /-]*(?P<year>\d{4})').reset_index()
len(pattern3)

34

In [92]:
#20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
#r'((?:\d{1,2} ))?((?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)[a-z]*[?:, -]* )(\d{4})'
pattern4_1 = df.str.extractall(r'(?P<date>\d{1,2})\s(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[?:., /-]*(?P<year>\d{4})').reset_index()
#len(pattern4_1)

In [93]:
#Feb 2009; Sep 2009; Oct 2010
pattern4_2 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[?:., /-]*(?P<year>\d{4})').reset_index()
pattern4 = pd.concat([pattern4_1,pattern4_2])
pattern4.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates
pattern4.loc[pattern4['date'].isnull(), ['date']] = 1
len(pattern4) #len(pattern4['level_0'].unique())

184

In [94]:
#6/2008; 12/2009  (\d{1,2})[/](\d{4})
pattern2 = df.str.extractall(r'(?P<month>\d{1,2})[/](?P<year>\d{4})').reset_index()
#print(len(pattern2)) #138
for i in list(pattern1_2['level_0'].unique()):
    pattern2.loc[pattern2['level_0'] == i, ['level_0']] = np.nan
pattern2.dropna(subset = ['level_0'], how = 'any', axis = 0, inplace = True)
#pattern2.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates
pattern2['date'] = 1
len(pattern2['level_0'].unique()), len(pattern2)

(112, 112)

In [95]:
#df.str.findall(r'(?P<month>%s|%s)[ .,/-]+(?P<year>\d{4})[() .,/?!;:-]{1,}\b'  
#                    %(month_str, full_mon))

In [96]:
#combine data and check whether there are any duplicates

combined_dates = pattern1.append(pattern2).append(
                    pattern3).append(
                    pattern4)
len(combined_dates['level_0'].unique())

455

In [97]:
#final: 2009; 2010
pattern5_1 = df.str.extractall(r'[a-z]?[^0-9](?P<year>\d{4})[^0-9]').reset_index()
check_index = combined_dates['level_0'].unique()
for i in range(len(pattern5_1)):
    if pattern5_1['level_0'][i] in check_index:
        pattern5_1.drop(i, inplace = True)
pattern5_2 = df.str.extractall(r'^[\"., ()-]*(?P<year>\d{4})[^0-9]').reset_index()
pattern5 = pd.concat([pattern6_1, pattern6_2])
pattern5.drop_duplicates(subset='level_0', inplace = True)
pattern5['month'] = 1
pattern5['date'] = 1
len(pattern5), len(pattern5['level_0'].unique())

(45, 45)

In [98]:
#Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
#pattern7 = df.str.extractall(r'(?P<month>%s)[a-z]*[ .,/-]+(?P<date>\d{1,2})%s{2}[ .,/-]+(?P<year>\d{4})' 
#                             %(month_str, date_str))
#len(pattern7) #no such type

In [99]:
combined_dates = combined_dates.append(pattern5)
#len(combined_dates)

In [100]:
from datetime import datetime

combined_dates['m-d-y'] =pd.to_datetime(combined_dates['month'].apply(str)+'/'+
                                        combined_dates['date'].apply(str)+'/'+
                                        combined_dates['year'].apply(str))
combined_dates = combined_dates.set_index('level_0').sort_index()
combined_dates['time_rank'] = 0

In [112]:
sorted_date = sorted(combined_dates['m-d-y'])
for i, v in enumerate(sorted_date):
    combined_dates.loc[combined_dates['m-d-y'] == v, ['time_rank']] = i+1
    
answer = combined_dates['time_rank']
#answer
#combined_dates

In [113]:
def date_sorter():
    '''
    Here is a list of some of the variants you might encounter in this dataset:
    
    <<Data structure>>
    04/20/2009; 04/20/09; 4/20/09; 4/3/09 <done>
    Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; <done>
    20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009 <done>
    Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009 <done>
    Feb 2009; Sep 2009; Oct 2010 <done>
    6/2008; 12/2009 <done>
    2009; 2010
    Once you have extracted these date patterns from the text, 
    the next step is to sort them in ascending chronological order accoring to the following rules:
    
    <<Data manipulation>>
    Assume all dates in xx/xx/xx format are mm/dd/yy
    Assume all dates where year is encoded in only two digits are years from the 1900's 
    (e.g. 1/5/89 is January 5th, 1989)
    If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
    If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
    Watch out for potential typos as this is a raw, real-life derived dataset.
    With these rules in mind, find the correct date in each note and return a pandas Series 
    in chronological order of the original Series' indices.
    
    <<Output>>
    Return a Series with ranking, more recent, higher ranking, and vise versa.
    '''
    #deal with str repressed as follows: "04/20/2009; 04/20/09; 4/20/09; 4/3/09"
    pattern1_1 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<date>\d{1,2})[/-](?P<year>\d{2})\b').reset_index()
    pattern1_1['year'] = '19'+pattern1_1['year']
    pattern1_2 = df.str.extractall(r'(?P<month>\d{1,2})[/-](?P<date>\d{1,2})[/-](?P<year>\d{4})\b').reset_index()
    pattern1 = pd.concat([pattern1_1,pattern1_2])
    pattern1.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates
    
    #Mar-20-2009; Mar 20, 2009; March 20, 2009; Mar. 20, 2009; Mar 20 2009; together 

    pattern3 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[-.]*\s(?P<date>(?:\d{1,2}))[?:., /-]*(?P<year>\d{4})').reset_index()
    #20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
    pattern4_1 = df.str.extractall(r'(?P<date>\d{1,2})\s(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[?:., /-]*(?P<year>\d{4})').reset_index()
    #Feb 2009; Sep 2009; Oct 2010
    pattern4_2 = df.str.extractall(r'(?P<month>(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec))[a-z]*[?:., /-]*(?P<year>\d{4})').reset_index()
    pattern4 = pd.concat([pattern4_1,pattern4_2])
    pattern4.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates
    pattern4.loc[pattern4['date'].isnull(), ['date']] = 1
    
    #6/2008; 12/2009  (\d{1,2})[/](\d{4})
    pattern2 = df.str.extractall(r'(?P<month>\d{1,2})[/](?P<year>\d{4})').reset_index()
    #print(len(pattern2)) #138
    for i in list(pattern1_2['level_0'].unique()):
        pattern2.loc[pattern2['level_0'] == i, ['level_0']] = np.nan
    pattern2.dropna(subset = ['level_0'], how = 'any', axis = 0, inplace = True)
    #pattern2.drop_duplicates(subset='level_0', inplace = True) #keep first, drop later duplicates
    pattern2['date'] = 1
    
    #combine data and check whether there are any duplicates

    combined_dates = pattern1.append(pattern2).append(
                    pattern3).append(
                    pattern4)

    #final: 2009; 2010
    pattern5_1 = df.str.extractall(r'[a-z]?[^0-9](?P<year>\d{4})[^0-9]').reset_index()
    check_index = combined_dates['level_0'].unique()
    for i in range(len(pattern5_1)):
        if pattern5_1['level_0'][i] in check_index:
            pattern5_1.drop(i, inplace = True)
    pattern5_2 = df.str.extractall(r'^[\"., ()-]*(?P<year>\d{4})[^0-9]').reset_index()
    pattern5 = pd.concat([pattern6_1, pattern6_2])
    pattern5.drop_duplicates(subset='level_0', inplace = True)
    pattern5['month'] = 1
    pattern5['date'] = 1
    
    combined_dates = combined_dates.append(pattern5)

    combined_dates['m-d-y'] =pd.to_datetime(combined_dates['month'].apply(str)+'/'+
                                        combined_dates['date'].apply(str)+'/'+
                                        combined_dates['year'].apply(str))
    combined_dates = combined_dates.set_index('level_0').sort_index()
    combined_dates['time_rank'] = 0
    
    sorted_date = sorted(combined_dates['m-d-y'])
    for i, v in enumerate(sorted_date):
        combined_dates.loc[combined_dates['m-d-y'] == v, ['time_rank']] = i+1
    
    answer = combined_dates['time_rank']
    return answer

In [114]:
date_sorter()

level_0
0.0      273
1.0      171
2.0        3
3.0       56
4.0      313
5.0      100
6.0       89
7.0      229
8.0      181
9.0        1
10.0     169
11.0      49
12.0     335
13.0       8
14.0     237
15.0     459
16.0     131
17.0      69
18.0     330
19.0      77
20.0     333
21.0     105
22.0     235
23.0      62
24.0     160
25.0     132
26.0     228
27.0      67
28.0       5
29.0     191
        ... 
470.0    144
471.0    340
472.0    447
473.0     46
474.0      6
475.0    489
476.0    215
477.0    282
478.0    270
479.0    311
480.0    479
481.0     27
482.0    231
483.0    294
484.0    389
485.0    196
486.0     16
487.0    256
488.0     71
489.0    165
490.0    419
491.0    441
492.0    178
493.0     84
494.0    373
495.0     97
496.0    409
497.0    431
498.0    403
499.0    107
Name: time_rank, Length: 500, dtype: int64